<b>DATA ANALYTICS PROJECT<b>:

Group members: Jesse Reisner, Ryan Cree, Morgan McKee, Melanie Klock

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Austin_Animal_Center_Outcomes.csv')
data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
3,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
4,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black


In [3]:
data.shape

(116560, 12)

<b>DATA PREP:</b>

<i>Animal ID, and DateTime columns can be excluded from analysis because they are not pertinent to predicting a pet's outcome. DateTime is a column identical to MonthYear and therefore can be excluded.

In [16]:
data['Outcome Type'].isna().sum() #get rid of these rows

6

In [21]:
data['Outcome Type'].value_counts()

Adoption           51399
Transfer           34667
Return to Owner    20488
Euthanasia          7775
Died                1062
Rto-Adopt            593
Disposal             483
Missing               67
Relocate              20
Name: Outcome Type, dtype: int64

In [19]:
data['Outcome Subtype'].value_counts()

Partner                28909
Foster                  9005
Rabies Risk             3346
SCRP                    3211
Suffering               3051
Snr                     2544
In Kennel                547
Aggressive               533
Offsite                  383
Medical                  299
In Foster                269
At Vet                   203
Behavior                 154
Enroute                   75
Underage                  29
Court/Investigation       25
In Surgery                22
Possible Theft            16
Field                     11
Barn                       4
Customer S                 3
Prc                        1
Name: Outcome Subtype, dtype: int64

In [11]:
data = data.drop(columns = ['DateTime'])

<b> DATA EXPLORATION: </b>

<b> FEATURE ENGINEERING: </b>